In [1]:
:load ./learning-2022/files/LectureNotes/Sections/interpreter/Parsing

{-# LANGUAGE TypeSynonymInstances #-}
{-# LANGUAGE FlexibleInstances    #-}

### Syntax

In [2]:
data Register = ACC
              | NIL
              | IN
              | OUT
              | Port String
           -- non-addressable:
              | BAK
              | IP
              deriving (Show, Read, Eq)

data Operand = Reg Register
             | Con Int
             deriving (Read, Eq)

type Condition = Int -> Bool
type Label = String

data Operator = MOV Register
              | Mov Register -- runtime only 
              | SWP
           -- | SAV -> MOV ACC BAK
           -- | NOP -> ADD NIL
              | ADD
              | SUB
              | NEG
              | Jmp Condition Label -- parsing only
              | JRO Condition
           -- | JMP | JEZ | JNZ | JGZ | JLZ
              | HCF
              deriving (Show, Read, Eq)

data Operation = O Operator Operand
               deriving (Show, Read)


-- instances

instance Show Operand where
  show (Reg x) = show x
  show (Con x) = show x

instance Eq Condition where
  x == y = map x [(-1)..1] == map y [(-1)..1]

instance Show Condition where
  show c = case map c [(-1)..1] of
             [True, True, True ] -> "__"
             [False,True, False] -> "EZ"
             [True, False,True ] -> "NZ"
             [False,False,True ] -> "GZ"
             [True, False,False] -> "LZ"

instance Read Condition where
  readsPrec _ _ = []

In [3]:
import qualified Data.Map as Map

type State = Register -> Int

data Mode = Default
          | Write Int Register
          deriving Show

data Node = Node [Operation] State Mode


instance Show Node where
  show (Node os s m) = show os

data Program = P (Map.Map String Node) [Int] [Int]

instance Show Program where
  show (P ns is os) = show ns ++ show is ++ show os

### Parsing

In [4]:
import Text.Read
import Data.Maybe
import Control.Monad
import Data.Either

import Parsing

In [5]:
goodchar :: Parser Char
goodchar = sat (`notElem` " :#\n")

word :: Parser String
word = token $ many goodchar

name :: Parser String
name = token $ do char '@'
                  word

register :: Parser Register
register = do xs <- word
              guard $ isJust (readMaybe xs :: Maybe Register)
              return (read xs :: Register)
       <|> Port <$> name

operand :: Parser Operand
operand =  Reg <$> register
       <|> Con <$> integer

operation :: Parser Operation
operation =  do symbol "MOV"
                o <- operand
                r <- register
                return $ O (MOV r) o
         <|> do symbol "SAV"
                return $ O (MOV BAK) (Reg ACC)
         <|> do symbol "NOP" 
                return $ O ADD (Reg NIL)
         <|> do symbol "ADD"
                O ADD <$> operand
         <|> do symbol "SUB"
                O SUB <$> operand
         <|> do symbol "JMP"
                w <- word
                return $ O (Jmp (const True) w) (Reg NIL)
         <|> do symbol "JEZ"
                w <- word
                return $ O (Jmp (==0)        w) (Reg NIL)
         <|> do symbol "JNZ"
                w <- word
                return $ O (Jmp (/=0)        w) (Reg NIL)
         <|> do symbol "JGZ"
                w <- word
                return $ O (Jmp (>0)         w) (Reg NIL)
         <|> do symbol "JLZ"
                w <- word
                return $ O (Jmp (<0)         w) (Reg NIL)
         <|> do symbol "JRO"
                O (JRO (const True)) <$> operand
         <|> do xs <- word
                guard $ isJust (readMaybe xs :: Maybe Operator)
                return $ O (read xs :: Operator) (Reg NIL)

                
label :: Parser Label
label = token $
          do w <- word
             char ':'
             return w

node :: Parser [Either Operation Label]
node = many $
        Left  <$> operation
    <|> Right <$> label

process :: [Either Operation Label] -> (Label -> Int) -> [Operation] -> [Operation]
process [] f os = p os
  where p [] = []
        p xs = p (init xs) ++ [q (last xs)]
          where q (O (Jmp c s) _) = O (JRO c) (Con (f s - length (init xs)))
                q x         = x
process (Left  o : es) f n = process es f (n ++ [o])
process (Right l : es) f n = process es f' n
  where f' l' | l' == l   = if all isRight es then 0 else length n
              | otherwise = f l'

parseNode :: String -> [Operation]
parseNode xs = case parse node xs of
                 [(p , [])] -> process p (const undefined) []
                 [(_ , s)]  -> error ("syntax: unparsed string " ++ s)
                 _          -> error "syntax: failed to parse"
                 
makeNode :: String -> Node
makeNode xs = Node (parseNode xs) (const 0) Default

In [6]:
namedNode :: Parser (String, [Either Operation Label])
namedNode = do x <- name
               y <- node
               return (x, y) 
           
program :: Parser [(String, [Either Operation Label])]
program = many namedNode

processProgram :: [(String, [Either Operation Label])] -> Map.Map String Node
processProgram = foldr (\ x -> Map.insert (fst x) (Node (process (snd x) (const undefined) []) (const 0) Default)) Map.empty

parseProgram :: String -> Program
parseProgram x = P ((processProgram . fst . head . parse program) x) [] []

### Interpreting

In [7]:
clamp :: Int -> Int -> Int -> Int
clamp l h x = max l $ min h x

update :: Register -> Int -> Node -> Node
update NIL _ n = n
update r   x (Node os s m) = Node os s' m
  where s' r' | r' == r   = clamp (-999) 999 x
              | otherwise = s r'

get :: Register -> Node -> Int
get r (Node os s _) = s r

len :: Node -> Int
len (Node os _ _) = length os

current :: Node -> Operation
current (Node os s Default)     = os !! s IP
current (Node os _ (Write i r)) = O (Mov r) (Con i)

mode :: Mode -> Node -> Node
mode m (Node os s _) = Node os s m

inc :: Node -> Node
inc n = mode Default $ update IP ((get IP n + 1) `mod` len n) n

jump :: Int -> Node -> Node
jump i n = update IP (clamp 0 (len n-1) i) n

eval :: Operand -> Node -> Int
eval (Reg r) s = get r s
eval (Con x) _ = x

exec :: Node -> Node
exec n = e' (current n) where
  e' :: Operation -> Node
  e' (O a b) = e a (eval b n) n
    
e :: Operator -> Int -> Node -> Node
e (MOV r) o n = if r `elem` [ACC, NIL, BAK] then update r o $ inc n else mode (Write o r) n
e (Mov _) _ n = n
e  SWP    _ n = update BAK (get ACC n) $ update ACC (get BAK n) $ inc n
e  ADD    o n = update ACC (get ACC n + o) $ inc n
e  SUB    o n = update ACC (get ACC n - o) $ inc n
e  NEG    _ n = update ACC (negate $ get ACC n) $ inc n
e (JRO c) o n = if c $ get ACC n then jump (get IP n + o) n else inc n
e  HCF    _ _ = error "halted and caught fire"

run :: Node -> Int -> Int
run n i = get OUT $ iterate exec (update IN i n) !! 1000

In [8]:
padRight :: a -> Int -> [a] -> [a]
padRight  c n xs = take n $ xs ++ repeat c

log :: Node -> Int -> Int -> String
log n i c = concatMap (("\n"++) . d) (take c $ iterate exec (update IN i n))

d :: Node -> String
d n = show (get IP n) ++ ": " ++ padRight ' ' 28 (show (current n)) ++ " | " ++ show (get ACC n) ++ " (" ++ show (get BAK n) ++ ")"

In [9]:
nodes :: Program -> [Node]
nodes (P ns _ _) = Map.elems ns

keys :: Program -> [String]
keys (P ns _ _) = Map.keys ns

ins :: Program -> [Int]
ins (P _ is _) = is

outs :: Program -> [Int]
outs (P _ _ os) = os

operand :: Node -> Operand
operand n = f $ current n 
  where f (O _ o) = o

operator :: Node -> Operator
operator n = f $ current n
  where f (O o _) = o

checkAnyRead :: Register -> Program -> Bool
checkAnyRead r (P ns _ _) = any ((== Reg r) . operand) (Map.elems ns)

checkRead :: Register -> Register -> Program -> Bool
checkRead (Port s) r (P ns _ _) = operand (ns Map.! s) == Reg r
checkRead OUT _ _ = True

asdf :: Operand -> Int
asdf (Con x) = x

readAny :: Register -> Program -> [Int]
readAny r p = [asdf $ operand a | a <- nodes p, operator a == Mov OUT]

eval' :: Register -> Operand -> Node -> Program -> Maybe Int
eval' _ (Con x) _ _ = Just x
eval' _ (Reg IN) _ (P _ (i:is) _) = Just i
eval' _ (Reg IN) _ (P _ []     _) = Nothing
eval' r (Reg (Port x)) _ (P ns _ _) = case current (ns Map.! x) of
                                        O (Mov r) (Con y) -> Just y
                                        _                 -> Nothing
eval' _ (Reg r) n _ = Just $ get r n
                                        
step :: Program -> Program
step (P ns is os) = P (Map.fromList [f n | n <- Map.toList ns]) (if not (null is) && checkAnyRead IN (P ns is os) then tail is else is) (os ++ readAny OUT (P ns is os))where
  f (k, v) = case (operator v, eval' (Port k) (operand v) v (P ns is os)) of
    (Mov r, _)  -> if checkRead r (Port k) (P ns is os) then (k, inc v) else (k, v)
    (o, Just x) -> (k, e o x v)
    _           -> (k, v)

input :: [Int] -> Program -> Program
input is (P ns _ os) = P ns is os

log' :: Program -> [Int] -> Int -> String
log' p is c = unlines [concatMap ((++"     ") . d) (f a) ++ show (ins a) ++ show (outs a) | a <- take c $ iterate step $ input is p]
  where f (P ns _ _) = Map.elems ns

In [10]:
biggie = unlines
  ["@1            ",
   "   MOV 0 ACC  ",
   "   SUB IN     ",
   "&: ADD @2     ",
   "   MOV -2 @2  ",
   "   JLZ &      ",
   "   MOV @2 @2  ",
   "   MOV ACC @2 ",
   "              ",
   "@2            ",
   "&: MOV ACC @1 ",
   "   SUB @1     ",
   "   JGZ &      ",
   "   MOV @1 OUT "]
   
b = parseProgram biggie

putStr $ log' b [1,4] 48

0: O (MOV ACC) 0                | 0 (0)     0: O (MOV (Port "1")) ACC       | 0 (0)     [1,4][]
1: O SUB IN                     | 0 (0)     0: O (Mov (Port "1")) 0         | 0 (0)     [1,4][]
2: O ADD Port "2"               | -1 (0)     0: O (Mov (Port "1")) 0         | 0 (0)     [4][]
3: O (MOV (Port "2")) -2        | -1 (0)     1: O SUB Port "1"               | 0 (0)     [4][]
3: O (Mov (Port "2")) -2        | -1 (0)     1: O SUB Port "1"               | 0 (0)     [4][]
4: O (JRO LZ) -2                | -1 (0)     2: O (JRO GZ) -2                | 2 (0)     [4][]
2: O ADD Port "2"               | -1 (0)     0: O (MOV (Port "1")) ACC       | 2 (0)     [4][]
2: O ADD Port "2"               | -1 (0)     0: O (Mov (Port "1")) 2         | 2 (0)     [4][]
3: O (MOV (Port "2")) -2        | 1 (0)     1: O SUB Port "1"               | 2 (0)     [4][]
3: O (Mov (Port "2")) -2        | 1 (0)     1: O SUB Port "1"               | 2 (0)     [4][]
4: O (JRO LZ) -2                | 1 (0)     2: O (

### Testing

In [11]:
example1 = 
  unlines [
    "  MOV IN  ACC",
    "  SAV        ",
    "  MOV 1   ACC",
    "             ",
    "L:SWP        ",
    "  JEZ E      ",
    "  JLZ F      ",
    "  SUB 1      ",
    "  SWP        ",
    "  ADD ACC    ",
    "  JMP L      ",
    "             ",
    "F:HCF        ",
    "             ",
    "E:SWP        ",
    "  MOV ACC OUT"]

putStr example1

  MOV IN  ACC
  SAV        
  MOV 1   ACC
             
L:SWP        
  JEZ E      
  JLZ F      
  SUB 1      
  SWP        
  ADD ACC    
  JMP L      
             
F:HCF        
             
E:SWP        
  MOV ACC OUT

In [12]:
parsed1 = makeNode example1
parsed1

[O (MOV ACC) IN,O (MOV BAK) ACC,O (MOV ACC) 1,O SWP NIL,O (JRO EZ) 7,O (JRO LZ) 5,O SUB 1,O SWP NIL,O ADD ACC,O (JRO __) -6,O HCF NIL,O SWP NIL,O (MOV OUT) ACC]

In [13]:
map (run parsed1) [0..11]

[0,0,0,0,0,0,0,0,0,0,0,0]

In [14]:
putStr $ log parsed1 4 39


0: O (MOV ACC) IN               | 0 (0)
1: O (MOV BAK) ACC              | 4 (0)
2: O (MOV ACC) 1                | 4 (4)
3: O SWP NIL                    | 1 (4)
4: O (JRO EZ) 7                 | 4 (1)
5: O (JRO LZ) 5                 | 4 (1)
6: O SUB 1                      | 4 (1)
7: O SWP NIL                    | 3 (1)
8: O ADD ACC                    | 1 (3)
9: O (JRO __) -6                | 2 (3)
3: O SWP NIL                    | 2 (3)
4: O (JRO EZ) 7                 | 3 (2)
5: O (JRO LZ) 5                 | 3 (2)
6: O SUB 1                      | 3 (2)
7: O SWP NIL                    | 2 (2)
8: O ADD ACC                    | 2 (2)
9: O (JRO __) -6                | 4 (2)
3: O SWP NIL                    | 4 (2)
4: O (JRO EZ) 7                 | 2 (4)
5: O (JRO LZ) 5                 | 2 (4)
6: O SUB 1                      | 2 (4)
7: O SWP NIL                    | 1 (4)
8: O ADD ACC                    | 4 (1)
9: O (JRO __) -6                | 8 (1)
3: O SWP NIL                    | 8 (1)

In [15]:
run parsed1 (-1)

: 

In [16]:
example2 =
  unlines [
    "  MOV 20 ACC ",
    "  JRO IN     ",
    "  JMP E      ",
    "             ",
    "2:SUB 10     ",
    "  JMP E      ",
    "             ",
    "4:SUB 1      ",
    "  JMP E      ",
    "             ",
    "6:ADD 1      ",
    "  JMP E      ",
    "             ",
    "8:ADD 10     ",
    "             ",
    "E:MOV ACC OUT"]
           
putStr example2

  MOV 20 ACC 
  JRO IN     
  JMP E      
             
2:SUB 10     
  JMP E      
             
4:SUB 1      
  JMP E      
             
6:ADD 1      
  JMP E      
             
8:ADD 10     
             
E:MOV ACC OUT

In [17]:
parsed2 = makeNode example2
parsed2

[O (MOV ACC) 20,O (JRO __) IN,O (JRO __) 8,O SUB 10,O (JRO __) 6,O SUB 1,O (JRO __) 4,O ADD 1,O (JRO __) 2,O ADD 10,O (MOV OUT) ACC]

In [18]:
map (run parsed2) [1..11]

[0,0,0,0,0,0,0,0,0,0,0]